# **데이터 수집 : 코스피 개별 종목의 종목 코드로 재무정보 가져오기**
- 종목코드 -> 고유번호 -> 법인등록번호 -> 재무정보
1. '금융감독원_고유번호'를 통해 '종목 코드'로 '고유번호' 가져오기
    - [**금융감독원_고유번호**](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018)
2. '금융감독원_공시정보_기업개황'에서 '고유번호'로 '법인등록번호' 가져오기
    - [**금융감독원_공시정보_기업개황**](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019002)
    - KOSPI200 기업 목록 활용
3. 가져온 '법인등록번호'으로 재무정보 가져오기
    - [**금융위원회_기업 재무정보**](https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15043459)
---
- [DART](https://dart.fss.or.kr/main.do) : 금융감독원에서 운영하는 기업정보전자공시시스템
- FSC : 금융위원회

In [35]:
import json
import os
import sys
from glob import glob
from io import BytesIO
from zipfile import ZipFile
import time

import numpy as np
import pandas as pd
import requests
import xmltodict
from tqdm import tqdm
from bs4 import BeautifulSoup

sys.path.append("../import")
from gitig_auth import authKey

authKey_dart = authKey["dart"]
authKey_fscfsi = authKey["fsc_finaStatInfo"]
data_path = "../data/"

# 분석할 종목코드 목록

# 1. 종목코드 -> 고유번호
- '금융감독원_고유번호'를 통해 '종목 코드'로 '고유번호' 가져오기
  - [**금융감독원_고유번호**](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018)

## Data API -> df_cc

In [36]:
url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={authKey_dart}"
response = requests.get(url)
if response.status_code == 200:
    with ZipFile(BytesIO(response.content)) as df_cc:
        df_cc = df_cc.read("CORPCODE.xml")
        df_cc = pd.read_xml(df_cc)

    display(df_cc.head())
else:
    print(response.status_code)

,corp_code,corp_name,stock_code,modify_date
0,434003,다코,NaN,20170630
1,434456,일산약품,NaN,20170630
2,430964,굿앤엘에스,NaN,20170630
3,432403,한라판지,NaN,20170630
4,388953,크레디피아제이십오차유동화전문회사,NaN,20170630


## df_cc 전처리

In [37]:
# stock_code가 없는 행 제거
df_cc = df_cc.dropna(subset=["stock_code"])
# code 글자수
df_cc["corp_code"] = df_cc["corp_code"].astype(int).astype(str).apply(lambda x: x.zfill(8))
df_cc["stock_code"] = df_cc["stock_code"].astype(int).astype(str).apply(lambda x: x.zfill(6))
# 컬럼 순서 설정
df_cc = df_cc[["stock_code", "corp_code", "corp_name"]]
df_cc.head()

,stock_code,corp_code,corp_name
2009,036720,00260985,한빛네트
2021,040130,00264529,엔플렉스
2022,055000,00358545,동서정보기술
2784,032600,00231567,애드모바일
3889,037600,00247939,씨모스


## df_cc에서 분석 대상만 남게

In [38]:
df_ksp_stock = pd.read_csv(f"{data_path}KOSPI_STOCK.csv")
df_ksp_stock["종목코드"] = df_ksp_stock["종목코드"].astype(str).apply(lambda x: x.zfill(6))
df_ksp_stock.head()

,종목코드,종목명,현재가,대비,등락률,거래대금(원),상장시가총액(원)
0,025850,KPX화인케미칼,"17,900",50,0.28,"72,956,900","68,020,000,000"
1,071970,STX메탈,"5,660",0,0.00,"62,721,280","126,218,000,000"
2,017900,광전자,"3,195",-10,0.31,"2,235,709,160","185,130,322,785"
3,001210,금호전기,"25,050",550,2.24,"657,408,900","173,247,077,550"
4,084010,대한제강,"6,440",130,2.06,"203,772,180","154,592,019,680"


In [39]:
list_stock = set(df_ksp_stock["종목코드"].to_list())    # KOSPI200 종목코드 리스트
df_cc.drop(df_cc[~df_cc['stock_code'].isin(list_stock)].index, inplace=True)
df_cc

,stock_code,corp_code,corp_name
10912,003640,00140380,유니온스틸
13183,064420,00399773,케이피케미칼
14065,053000,00375302,우리금융지주
25841,068870,00423609,LG생명과학
26119,037620,00311030,미래에셋증권
...,...,...,...
96988,079980,00362238,휴비스
96992,010120,00105855,엘에스일렉트릭
96998,005930,00126380,삼성전자
97061,096760,00632304,JW홀딩스


## df_cc 영속화 : DART_corpCode.csv

In [40]:
# DART_corpCode
df_cc.to_csv(f"{data_path}DART_corpCode.csv", index=False)

# 고유번호 -> 법인등록번호
2. '금융감독원_공시정보_기업개황'에서 '고유번호'로 '법인등록번호' 가져오기
    - [**금융감독원_공시정보_기업개황**](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019002)
    - KOSPI200 기업 목록 활용

### 파일 불러오기

In [41]:
df_cc = pd.read_csv(f"{data_path}DART_corpCode.csv", index_col=False)
# code 글자수
df_cc["corp_code"] = df_cc["corp_code"].astype(int).astype(str).apply(lambda x: x.zfill(8))
df_cc["stock_code"] = df_cc["stock_code"].astype(int).astype(str).apply(lambda x: x.zfill(6))
df_cc

,stock_code,corp_code,corp_name
0,003640,00140380,유니온스틸
1,064420,00399773,케이피케미칼
2,053000,00375302,우리금융지주
3,068870,00423609,LG생명과학
4,037620,00311030,미래에셋증권
...,...,...,...
324,079980,00362238,휴비스
325,010120,00105855,엘에스일렉트릭
326,005930,00126380,삼성전자
327,096760,00632304,JW홀딩스


## 함수 : 종목코드 -> 법인등록번호

In [42]:
# 종목코드 -> 고유번호
def stockCode_to_corpCode(stock_code, df=df_cc):
    cropCode = df[df["stock_code"] == stock_code]["corp_code"].values[0]
    return cropCode


# 고유번호 -> 법인등록번호
def corpCode_to_jurirNo(corp_code, authKey=authKey_dart):

    url = "https://opendart.fss.or.kr/api/company.json"
    params = {"crtfc_key": authKey, "corp_code": corp_code}

    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()["jurir_no"]
    else:
        print(response.status_code)

    time.sleep(0.01)


# 종목코드 -> 고유번호 -> 법인등록번호
def stockCode_to_jurirNo(stock_code, df=df_cc, authKey=authKey_dart):

    cropCode = stockCode_to_corpCode(stock_code, df)
    jurirNo = corpCode_to_jurirNo(cropCode, authKey)
    return jurirNo


# 함수 테스트
stock_code = "005930"
cropCode = stockCode_to_corpCode(stock_code)
jurirNo = stockCode_to_jurirNo(stock_code)
print(cropCode)
print(jurirNo)

00126380
1301110006246


In [43]:
def temp(stock_code):
    jurirNo = stockCode_to_jurirNo(stock_code)
    return jurirNo
df_cc["jurir_no"] = df_cc["stock_code"].map(temp)
df_cc

,stock_code,corp_code,corp_name,jurir_no
0,003640,00140380,유니온스틸,1101110041501
1,064420,00399773,케이피케미칼,2301110082112
2,053000,00375302,우리금융지주,1101112202797
3,068870,00423609,LG생명과학,1101112581183
4,037620,00311030,미래에셋증권,1101111821423
...,...,...,...,...
324,079980,00362238,휴비스,1101112102070
325,010120,00105855,엘에스일렉트릭,1101110520076
326,005930,00126380,삼성전자,1301110006246
327,096760,00632304,JW홀딩스,1101113710468


## df_cc 영속화2 : DART_corpCode.csv

In [44]:
# DART_corpCode
df_cc.to_csv(f"{data_path}DART_corpCode.csv", index=False)

# 법인등록번호 -> 재무정보
금융위원회_기업 재무정보
- 요청 URL : http://apis.data.go.kr/1160100/service/GetFinaStatInfoService/getSummFinaStat

## 파일 불러오기

In [45]:
df_cc = pd.read_csv(f"{data_path}DART_corpCode.csv", index_col=False)
# code 글자수
df_cc["corp_code"] = df_cc["corp_code"].astype(int).astype(str).apply(lambda x: x.zfill(8))
df_cc["stock_code"] = df_cc["stock_code"].astype(int).astype(str).apply(lambda x: x.zfill(6))
df_cc["jurir_no"] = df_cc["jurir_no"].astype(str).apply(lambda x: x.zfill(13))
df_cc

,stock_code,corp_code,corp_name,jurir_no
0,003640,00140380,유니온스틸,1101110041501
1,064420,00399773,케이피케미칼,2301110082112
2,053000,00375302,우리금융지주,1101112202797
3,068870,00423609,LG생명과학,1101112581183
4,037620,00311030,미래에셋증권,1101111821423
...,...,...,...,...
324,079980,00362238,휴비스,1101112102070
325,010120,00105855,엘에스일렉트릭,1101110520076
326,005930,00126380,삼성전자,1301110006246
327,096760,00632304,JW홀딩스,1101113710468


In [46]:
# 함수
def Get_FinaStatInfo(
    crno,
    authKey,
    bizYear="",
    numOfRows="",
    pageNo="",
    url="http://apis.data.go.kr/1160100/service/GetFinaStatInfoService/getSummFinaStat",
):

    params = {
        "serviceKey": authKey,
        "numOfRows": numOfRows,
        "pageNo": numOfRows,
        "resultType": "json",
        "crno": crno,
        "bizYear": bizYear,
    }

    response = requests.get(url, params=params)
    time.sleep(0.01)

    if response.status_code == 200:
        return response.json()["response"]["body"]["items"]["item"]

    else:
        print(response.status_code)

In [48]:
df_fsi = pd.DataFrame()

for jurirNo in tqdm(df_cc["jurir_no"].values[:]):
    data_json = Get_FinaStatInfo(jurirNo, authKey=authKey_fscfsi)
    tmp = pd.json_normalize(data_json)
    df_fsi = pd.concat([df_fsi, tmp], axis=0, sort=False)

df_fsi

100%|██████████| 329/329 [02:05<00:00,  2.63it/s]


,basDt,crno,bizYear,fnclDcd,fnclDcdNm,enpSaleAmt,enpBzopPft,iclsPalClcAmt,enpCrtmNpf,enpTastAmt,enpTdbtAmt,enpTcptAmt,enpCptlAmt,fnclDebtRto
0,20151231,1101112581183,2015,ifrs_ConsolidatedMember,연결요약재무제표,450526355034,25201939449,13894239541,11371548367,706980326807,449358007227,257622319580,84066030000,174.4251072499
1,20151231,1101112581183,2015,ifrs_SeparateMember,별도요약재무제표,435446816306,26136971686,15021017434,12443782196,703705604242,445384074854,258321529388,84066030000,172.4146167411
0,20150331,1748110000151,2015,ifrs_ConsolidatedMember,연결요약재무제표,207890217005,-871462412,-5660470477,-4315628598,727951594676,430331568399,297620026277,0,144.590931525
1,20150331,1748110000151,2015,ifrs_SeparateMember,별도요약재무제표,175932552323,-2615227674,-6690401088,-5677659364,669744438135,397542364257,272202073878,0,146.0467800973
2,20160331,1748110000151,2016,ifrs_ConsolidatedMember,연결요약재무제표,856457055850,48660898849,33307151671,21854084841,713772955053,397204546704,316568408349,0,125.4719473669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,20171231,1101110003733,2017,ifrs_SeparateMember,별도요약재무제표,286386535237,19514857445,19981045677,13814407827,689594455044,66157192173,623437262871,8450000000,10.6116839838
6,20181231,1101110003733,2018,ifrs_ConsolidatedMember,연결요약재무제표,864585835647,32798360071,74914386097,51472655804,919690198948,170684939463,749005259485,8450000000,22.7882164112
7,20181231,1101110003733,2018,ifrs_SeparateMember,별도요약재무제표,305132945611,19773999199,55139663239,34676568703,707393585361,61768795388,645624789973,8450000000,9.5672899101
8,20191231,1101110003733,2019,ifrs_ConsolidatedMember,연결요약재무제표,933866093002,23483614049,25514184242,16993288734,1005108274460,245259980306,759848294154,8450000000,32.277493046


## df_fsi 영속화 : FSC_finaStatInfo.csv

In [49]:
df_fsi.to_csv(f"{data_path}/FSC_finaStatInfo.csv", index=False)

# 병합 및 전처리

## 파일 불러오기

In [50]:
df_cc = pd.read_csv(f"{data_path}DART_corpCode.csv")
# code 글자수
df_cc["corp_code"] = df_cc["corp_code"].astype(int).astype(str).apply(lambda x: x.zfill(8))
df_cc["stock_code"] = df_cc["stock_code"].astype(int).astype(str).apply(lambda x: x.zfill(6))
df_cc["jurir_no"] = df_cc["jurir_no"].astype(str).apply(lambda x: x.zfill(13))
display(df_cc.head())


df_fsi = pd.read_csv(f"{data_path}FSC_finaStatInfo.csv")
df_fsi["crno"] = df_fsi["crno"].astype(str).apply(lambda x: x.zfill(13))
display(df_fsi.head())

,stock_code,corp_code,corp_name,jurir_no
0,003640,00140380,유니온스틸,1101110041501
1,064420,00399773,케이피케미칼,2301110082112
2,053000,00375302,우리금융지주,1101112202797
3,068870,00423609,LG생명과학,1101112581183
4,037620,00311030,미래에셋증권,1101111821423


,basDt,crno,bizYear,fnclDcd,fnclDcdNm,enpSaleAmt,enpBzopPft,iclsPalClcAmt,enpCrtmNpf,enpTastAmt,enpTdbtAmt,enpTcptAmt,enpCptlAmt,fnclDebtRto
0,20151231,1101112581183,2015,ifrs_ConsolidatedMember,연결요약재무제표,450526355034,25201939449,13894239541,11371548367,706980326807,449358007227,257622319580,84066030000,174.425107
1,20151231,1101112581183,2015,ifrs_SeparateMember,별도요약재무제표,435446816306,26136971686,15021017434,12443782196,703705604242,445384074854,258321529388,84066030000,172.414617
2,20150331,1748110000151,2015,ifrs_ConsolidatedMember,연결요약재무제표,207890217005,-871462412,-5660470477,-4315628598,727951594676,430331568399,297620026277,0,144.590932
3,20150331,1748110000151,2015,ifrs_SeparateMember,별도요약재무제표,175932552323,-2615227674,-6690401088,-5677659364,669744438135,397542364257,272202073878,0,146.046780
4,20160331,1748110000151,2016,ifrs_ConsolidatedMember,연결요약재무제표,856457055850,48660898849,33307151671,21854084841,713772955053,397204546704,316568408349,0,125.471947


## 병합

In [51]:
df_fsi = pd.merge(df_fsi, df_cc, how='left', left_on="crno", right_on="jurir_no")
df_fsi

,basDt,crno,bizYear,fnclDcd,fnclDcdNm,enpSaleAmt,enpBzopPft,iclsPalClcAmt,enpCrtmNpf,enpTastAmt,enpTdbtAmt,enpTcptAmt,enpCptlAmt,fnclDebtRto,stock_code,corp_code,corp_name,jurir_no
0,20151231,1101112581183,2015,ifrs_ConsolidatedMember,연결요약재무제표,450526355034,25201939449,13894239541,11371548367,706980326807,449358007227,257622319580,84066030000,174.425107,068870,00423609,LG생명과학,1101112581183
1,20151231,1101112581183,2015,ifrs_SeparateMember,별도요약재무제표,435446816306,26136971686,15021017434,12443782196,703705604242,445384074854,258321529388,84066030000,172.414617,068870,00423609,LG생명과학,1101112581183
2,20150331,1748110000151,2015,ifrs_ConsolidatedMember,연결요약재무제표,207890217005,-871462412,-5660470477,-4315628598,727951594676,430331568399,297620026277,0,144.590932,008000,00148717,도레이케미칼,1748110000151
3,20150331,1748110000151,2015,ifrs_SeparateMember,별도요약재무제표,175932552323,-2615227674,-6690401088,-5677659364,669744438135,397542364257,272202073878,0,146.046780,008000,00148717,도레이케미칼,1748110000151
4,20160331,1748110000151,2016,ifrs_ConsolidatedMember,연결요약재무제표,856457055850,48660898849,33307151671,21854084841,713772955053,397204546704,316568408349,0,125.471947,008000,00148717,도레이케미칼,1748110000151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747,20171231,1101110003733,2017,ifrs_SeparateMember,별도요약재무제표,286386535237,19514857445,19981045677,13814407827,689594455044,66157192173,623437262871,8450000000,10.611684,001130,00113243,대한제분,1101110003733
2748,20181231,1101110003733,2018,ifrs_ConsolidatedMember,연결요약재무제표,864585835647,32798360071,74914386097,51472655804,919690198948,170684939463,749005259485,8450000000,22.788216,001130,00113243,대한제분,1101110003733
2749,20181231,1101110003733,2018,ifrs_SeparateMember,별도요약재무제표,305132945611,19773999199,55139663239,34676568703,707393585361,61768795388,645624789973,8450000000,9.567290,001130,00113243,대한제분,1101110003733
2750,20191231,1101110003733,2019,ifrs_ConsolidatedMember,연결요약재무제표,933866093002,23483614049,25514184242,16993288734,1005108274460,245259980306,759848294154,8450000000,32.277493,001130,00113243,대한제분,1101110003733


## 전처리

In [52]:
df_fsi['YMD'] = pd.to_datetime(df_fsi['basDt'], format='%Y%m%d')
df_fsi['Y'] = df_fsi['YMD'].dt.year
df_fsi['M'] = df_fsi['YMD'].dt.month
df_fsi['Q'] = df_fsi['YMD'].dt.quarter
df_fsi['YM'] = df_fsi['Y'].astype("str") + "-" + df_fsi['M'].astype("str")
df_fsi['YQ'] = df_fsi['Y'].astype("str") + "-" + df_fsi['Q'].astype("str")
df_fsi

,basDt,crno,bizYear,fnclDcd,fnclDcdNm,enpSaleAmt,enpBzopPft,iclsPalClcAmt,enpCrtmNpf,enpTastAmt,...,stock_code,corp_code,corp_name,jurir_no,YMD,Y,M,Q,YM,YQ
0,20151231,1101112581183,2015,ifrs_ConsolidatedMember,연결요약재무제표,450526355034,25201939449,13894239541,11371548367,706980326807,...,068870,00423609,LG생명과학,1101112581183,2015-12-31,2015,12,4,2015-12,2015-4
1,20151231,1101112581183,2015,ifrs_SeparateMember,별도요약재무제표,435446816306,26136971686,15021017434,12443782196,703705604242,...,068870,00423609,LG생명과학,1101112581183,2015-12-31,2015,12,4,2015-12,2015-4
2,20150331,1748110000151,2015,ifrs_ConsolidatedMember,연결요약재무제표,207890217005,-871462412,-5660470477,-4315628598,727951594676,...,008000,00148717,도레이케미칼,1748110000151,2015-03-31,2015,3,1,2015-3,2015-1
3,20150331,1748110000151,2015,ifrs_SeparateMember,별도요약재무제표,175932552323,-2615227674,-6690401088,-5677659364,669744438135,...,008000,00148717,도레이케미칼,1748110000151,2015-03-31,2015,3,1,2015-3,2015-1
4,20160331,1748110000151,2016,ifrs_ConsolidatedMember,연결요약재무제표,856457055850,48660898849,33307151671,21854084841,713772955053,...,008000,00148717,도레이케미칼,1748110000151,2016-03-31,2016,3,1,2016-3,2016-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747,20171231,1101110003733,2017,ifrs_SeparateMember,별도요약재무제표,286386535237,19514857445,19981045677,13814407827,689594455044,...,001130,00113243,대한제분,1101110003733,2017-12-31,2017,12,4,2017-12,2017-4
2748,20181231,1101110003733,2018,ifrs_ConsolidatedMember,연결요약재무제표,864585835647,32798360071,74914386097,51472655804,919690198948,...,001130,00113243,대한제분,1101110003733,2018-12-31,2018,12,4,2018-12,2018-4
2749,20181231,1101110003733,2018,ifrs_SeparateMember,별도요약재무제표,305132945611,19773999199,55139663239,34676568703,707393585361,...,001130,00113243,대한제분,1101110003733,2018-12-31,2018,12,4,2018-12,2018-4
2750,20191231,1101110003733,2019,ifrs_ConsolidatedMember,연결요약재무제표,933866093002,23483614049,25514184242,16993288734,1005108274460,...,001130,00113243,대한제분,1101110003733,2019-12-31,2019,12,4,2019-12,2019-4


In [53]:
dict_colReName = {
    "stock_code": "종목코드",
    "corp_name": "종목명",
    "basDt": "기준일자",
    "crno": "법인등록번호",
    "bizYear": "사업연도",
    "fnclDcd": "재무제표구분코드",
    "fnclDcdNm": "재무제표구분코드명",
    "enpSaleAmt": "기업매출금액",
    "enpBzopPft": "기업영업이익",
    "iclsPalClcAmt": "포괄손익계산금액",
    "enpCrtmNpf": "기업당기순이익",
    "enpTastAmt": "기업총자산금액",
    "enpTdbtAmt": "기업총부채금액",
    "enpTcptAmt": "기업총자본금액",
    "enpCptlAmt": "기업자본금액",
    "fnclDebtRto": "재무제표부채비율",
}

col = ['종목코드', '종목명', 'YMD', 'Y', "M", "Q", "YM", "YQ", '재무제표구분코드', '재무제표구분코드명', '기업매출금액', '기업영업이익', '포괄손익계산금액', '기업당기순이익', '기업총자산금액', '기업총부채금액', '기업총자본금액', '기업자본금액','재무제표부채비율']


df_fsi = df_fsi.rename(columns=dict_colReName)
df_fsi = df_fsi[col]
df_fsi

,종목코드,종목명,YMD,Y,M,Q,YM,YQ,재무제표구분코드,재무제표구분코드명,기업매출금액,기업영업이익,포괄손익계산금액,기업당기순이익,기업총자산금액,기업총부채금액,기업총자본금액,기업자본금액,재무제표부채비율
0,068870,LG생명과학,2015-12-31,2015,12,4,2015-12,2015-4,ifrs_ConsolidatedMember,연결요약재무제표,450526355034,25201939449,13894239541,11371548367,706980326807,449358007227,257622319580,84066030000,174.425107
1,068870,LG생명과학,2015-12-31,2015,12,4,2015-12,2015-4,ifrs_SeparateMember,별도요약재무제표,435446816306,26136971686,15021017434,12443782196,703705604242,445384074854,258321529388,84066030000,172.414617
2,008000,도레이케미칼,2015-03-31,2015,3,1,2015-3,2015-1,ifrs_ConsolidatedMember,연결요약재무제표,207890217005,-871462412,-5660470477,-4315628598,727951594676,430331568399,297620026277,0,144.590932
3,008000,도레이케미칼,2015-03-31,2015,3,1,2015-3,2015-1,ifrs_SeparateMember,별도요약재무제표,175932552323,-2615227674,-6690401088,-5677659364,669744438135,397542364257,272202073878,0,146.046780
4,008000,도레이케미칼,2016-03-31,2016,3,1,2016-3,2016-1,ifrs_ConsolidatedMember,연결요약재무제표,856457055850,48660898849,33307151671,21854084841,713772955053,397204546704,316568408349,0,125.471947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747,001130,대한제분,2017-12-31,2017,12,4,2017-12,2017-4,ifrs_SeparateMember,별도요약재무제표,286386535237,19514857445,19981045677,13814407827,689594455044,66157192173,623437262871,8450000000,10.611684
2748,001130,대한제분,2018-12-31,2018,12,4,2018-12,2018-4,ifrs_ConsolidatedMember,연결요약재무제표,864585835647,32798360071,74914386097,51472655804,919690198948,170684939463,749005259485,8450000000,22.788216
2749,001130,대한제분,2018-12-31,2018,12,4,2018-12,2018-4,ifrs_SeparateMember,별도요약재무제표,305132945611,19773999199,55139663239,34676568703,707393585361,61768795388,645624789973,8450000000,9.567290
2750,001130,대한제분,2019-12-31,2019,12,4,2019-12,2019-4,ifrs_ConsolidatedMember,연결요약재무제표,933866093002,23483614049,25514184242,16993288734,1005108274460,245259980306,759848294154,8450000000,32.277493


## df_fsi 영속화2 : FSC_finaStatInfo.csv

In [ ]:
df_fsi.to_csv(f"{data_path}/FSC_finaStatInfo.csv", index=False)